In [1]:
import pandas as pd
import numpy as np
import torch

from eugene.dataload._io import read_numpy, read

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

# Basic import
import eugene as eu
eu.__version__

Global seed set to 13


GPU is available: True
Number of GPUs: 1
Current GPU: 0
GPUs: NVIDIA GeForce RTX 2070


'0.0.6'

In [2]:
sdata = read_numpy("chr1.npy_ohe_seqs.npy", ohe=True)

In [3]:
eu.pp.train_test_split_sdata(sdata)

SeqData object modified:
    seqs_annot:
        + train_val


In [4]:
seq_len = 50
seq_dim = seq_len * 4
latent_dim = 300

In [5]:
generator = torch.nn.Sequential(
    eu.models.FCN(
        input_len=latent_dim,
        output_dim=seq_dim,
        fc_kwargs=dict(hidden_dims=[300])
    ),
    torch.nn.Tanh()
)
generator

Sequential(
  (0): FCN(
    (hp_metric): R2Score()
    (fcnet): BasicFullyConnectedModule(
      (module): Sequential(
        (0): Linear(in_features=1200, out_features=300, bias=True)
        (1): ReLU()
        (2): Linear(in_features=300, out_features=200, bias=True)
      )
    )
  )
  (1): Tanh()
)

In [6]:
discriminator = torch.nn.Sequential(
    eu.models.FCN(
        input_len=seq_len,
        output_dim=1,
        fc_kwargs=dict(hidden_dims=[256, 64]),
    ),
    torch.nn.Sigmoid()
)
discriminator

Sequential(
  (0): FCN(
    (hp_metric): R2Score()
    (fcnet): BasicFullyConnectedModule(
      (module): Sequential(
        (0): Linear(in_features=200, out_features=256, bias=True)
        (1): ReLU()
        (2): Linear(in_features=256, out_features=64, bias=True)
        (3): ReLU()
        (4): Linear(in_features=64, out_features=1, bias=True)
      )
    )
  )
  (1): Sigmoid()
)

In [7]:
model = eu.models.GAN(
    latent_dim=latent_dim, 
    generator=generator, 
    discriminator=discriminator,
    mode="gan",
    grad_clip=0.01,
    gen_lr=1e-3,
    disc_lr=1e-3,
)

In [8]:
eu.settings.batch_size = 256
eu.settings.dl_num_workers = 8
eu.settings.dl_pin_memory_gpu_training = True
eu.train.fit(
    model=model,
    sdata=sdata,
    early_stopping_callback=False,
    gpus=0,
    early_stopping_metric="val_generator_loss"
)

Global seed set to 13
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


No transforms given, assuming just need to tensorize.
No transforms given, assuming just need to tensorize.


c:\Users\Lab\anaconda3\envs\eugene\lib\site-packages\pytorch_lightning\trainer\trainer.py:1584: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(

  | Name          | Type       | Params
---------------------------------------------
0 | generator     | Sequential | 420 K 
1 | discriminator | Sequential | 68.0 K
---------------------------------------------
488 K     Trainable params
0         Non-trainable params
488 K     Total params
1.954     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 13


Training: 0it [00:00, ?it/s]

c:\Users\Lab\anaconda3\envs\eugene\lib\site-packages\pytorch_lightning\trainer\trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


---